# Exploring Text Generation Parameters with HuggingFace Pipeline

This notebook demonstrates how different generative parameters affect text generation using HuggingFace's `pipeline`. We will experiment with temperature, `max_new_tokens`, and different models to observe their effect on the output text's creativity, coherence, and legibility.

In [ ]:
!pip install transformers
!pip install torch

## 1. Load the Text Generation Pipeline

In [ ]:
from transformers import pipeline

# Load a default model (using GPT-2 here, as it’s smaller for initial runs)
generator = pipeline("text-generation", model="gpt2")

## 2. Baseline Text Generation (Default Parameters)

In [ ]:
prompt = "The future of AI is"
output = generator(prompt, max_new_tokens=50)
print(output[0]['generated_text'])

## 3. Experiment 1: Increasing Temperature (0.9)

In [ ]:
output = generator(prompt, max_new_tokens=50, temperature=0.9)
print(output[0]['generated_text'])

## 4. Experiment 2: Higher Temperature (1.2) and Fewer Tokens

In [ ]:
output = generator("In a world where robots have taken over", max_new_tokens=20, temperature=1.2)
print(output[0]['generated_text'])

## 5. Experiment 3: Switching to Another Model (`microsoft/Phi-3-mini-4k-instruct`)

In [ ]:
# Load a different model
generator = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")

# Generate text with the new model and adjusted temperature
output = generator("The future of AI is", max_new_tokens=50, temperature=0.7)
print(output[0]['generated_text'])

## 6. Summary of Results and Analysis

In this notebook, we explored how adjusting the temperature, `max_new_tokens`, and the model impacts the legibility and creativity of text generation. Here’s a quick summary:

- **Lower temperatures (e.g., 0.7)** tend to produce more predictable, coherent outputs.
- **Higher temperatures (1.0–1.2)** introduce more randomness, leading to creative but sometimes less coherent text.
- **Different models** provide varied outputs, with some focusing on more factual, instructional content.

By fine-tuning these parameters, you can adjust the output to better fit your specific needs, whether that’s creativity, factual accuracy, or coherence.